In [2]:
!pip install pytorch-tabnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.2 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from pytorch_tabnet.tab_model import TabNetRegressor
import torch


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_excel('/content/drive/MyDrive/my_data_DS317/train_update.xlsx')
df.drop(['gioitinh'], axis=1)
df.head()

,mssv,namsinh,gioitinh,drl,diem_tt,dtb_toankhoa,dtb_tichluy,sotc_tichluy,diemtbhk_1,diemtbhk_2,diemtbhk_3,diemtbhk_4,diemtbhk_5,diemtbhk_6,diemtbhk_7,diemtbhk_8
0,39B39397XPvAibaEXe/vGZ1xWLvpK7wMZlgBlCBs,19,0,75.198135,30.0,9.37,9.37,24,9.37,9.34,9.40,9.370000,9.370000,9.370000,9.370000,9.370000
1,3AB392AFXPvAibaEXe/NRwqjPAC0MKjf0QSiSLjM,17,0,85.000000,26.5,7.07,7.07,139,7.00,6.74,6.25,6.230000,7.150000,6.920000,6.970000,7.600000
2,BEC54CF0XPvAibaEXe/qZKhoIW7F1rbPON7lHJUG,19,0,82.000000,30.0,6.54,6.54,18,6.54,6.54,6.54,6.540000,6.540000,6.540000,6.540000,6.540000
3,43386D57XPvAibaEXe/nw3DhNQnHfzJZvI0o00SJ,20,0,75.655535,22.0,1.83,1.83,4,3.56,0.00,1.78,1.780000,1.780000,1.780000,1.780000,1.780000
4,8F1A725FXPvAibaEXe88DMpF1KL2K3HyZfgFjAif,19,0,100.000000,30.0,9.31,9.31,64,9.21,9.32,9.41,9.313333,9.313333,9.313333,9.313333,9.313333


In [6]:
df_test = pd.read_excel('/content/drive/MyDrive/my_data_DS317/test_update.xlsx')
df_test.drop(['gioitinh'], axis=1)
df_test.head()

,mssv,namsinh,gioitinh,drl,diem_tt,dtb_toankhoa,dtb_tichluy,sotc_tichluy,diemtbhk_1,diemtbhk_2,diemtbhk_3,diemtbhk_4,diemtbhk_5,diemtbhk_6,diemtbhk_7,diemtbhk_8
0,14EEA146XPvAibaEXe/2sgPA+OKAks3AV3zsZSFH,22,0,82.105913,26.762789,8.77,8.77,123,8.61,8.79,9.01,9.04,8.63,8.620,8.530,8.747143
1,4A2628C7XPvAibaEXe9urngX21O5XaWExeziRFls,22,1,82.105913,30.000000,8.42,8.42,130,9.11,8.64,7.78,8.60,8.45,8.300,9.300,8.597143
2,44634117XPvAibaEXe+WdrMfsV/1fSPAGTu3hQ5U,22,0,82.105913,22.350000,7.73,7.73,137,7.83,7.46,7.53,7.67,7.89,8.030,7.750,7.737143
3,8A7BA5ACXPvAibaEXe8FNnc+c+WuJHdW9KyIp7Nl,19,0,100.000000,30.000000,6.52,6.52,83,7.63,7.21,7.70,7.05,1.67,6.252,6.252,6.252000
4,A3A77666XPvAibaEXe/4WyTnp6RTwi1hkjCNtuCX,20,0,75.655535,24.250000,6.50,6.50,148,6.02,6.75,6.79,6.26,5.21,5.580,6.450,6.690000


In [7]:
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100 if np.all(y_true != 0) else np.nan
    mpe = np.mean((y_true - y_pred) / y_true) * 100 if np.all(y_true != 0) else np.nan
    r2 = r2_score(y_true, y_pred)
    return {"R2 Score": r2, "MSE": mse, "RMSE": rmse, "MAE": mae, "MAPE": mape, "MPE": mpe}

In [ ]:
# model=  TabNetRegressor(
#         n_d=32, n_a=32, n_steps=3,
#         #random_state=42
# )

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [8]:
from sklearn.model_selection import ParameterGrid
from pytorch_tabnet.tab_model import TabNetRegressor
import torch
import numpy as np

# Khởi tạo danh sách tham số khả quan
param_grid = {
    'n_d': [8, 32, 48],
    'n_a': [8, 32, 48],
    'n_steps': [5, 7],
}

grid = list(ParameterGrid(param_grid))

best_model = None
best_score = float('inf')

In [9]:
initial_columns = ['namsinh', 'gioitinh', 'dtb_toankhoa', 'dtb_tichluy', 'sotc_tichluy', 'diemtbhk_1']

# Duyệt qua các bộ tham số
for params in grid:
    # Khởi tạo mô hình TabNetRegressor với tham số
    model = TabNetRegressor(
        n_d=params['n_d'],
        n_a=params['n_a'],
        n_steps=params['n_steps'],
    )

    # Biến lưu kết quả
    results = []
    current_X_columns = initial_columns.copy()

    # Lặp qua từng cột làm biến mục tiêu (y)
    for i in range(len(initial_columns) + 1, len(df.columns) - 1):
        if i + 2 >= len(df.columns):
            break

        # Xác định cột mục tiêu (y)
        target_column = df.columns[i + 2]

        # Xác định tập huấn luyện và kiểm tra
        X_train = df[current_X_columns].values
        y_train = df[target_column].values

        X_test = df_test[current_X_columns].values
        y_test = df_test[target_column].values

        # Huấn luyện mô hình
        model.fit(
            X_train, y_train.reshape(-1, 1),
            max_epochs=100,
            patience=10,
        )

        # Dự đoán
        y_pred = model.predict(X_test)

        # Đánh giá hiệu suất
        mse = np.mean((y_test - y_pred.ravel()) ** 2)  # Mean Squared Error
        if mse < best_score:
            best_score = mse
            best_model = model

        # Ghi lại kết quả
        results.append({
            "Target Column": target_column,
            "MSE": mse,
            "Params": params
        })

        # Cập nhật X với cột mới
        current_X_columns.append(target_column)

# In kết quả tốt nhất
print(f"Best Score: {best_score}")
print(f"Best Parameters: {params}")


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 7.12708 |  0:00:01s
epoch 1  | loss: 1.53646 |  0:00:02s
epoch 2  | loss: 1.07986 |  0:00:03s
epoch 3  | loss: 1.03461 |  0:00:04s
epoch 4  | loss: 0.95405 |  0:00:05s
epoch 5  | loss: 0.95865 |  0:00:07s
epoch 6  | loss: 1.01314 |  0:00:09s
epoch 7  | loss: 0.9478  |  0:00:10s
epoch 8  | loss: 0.88482 |  0:00:12s
epoch 9  | loss: 0.84979 |  0:00:13s
epoch 10 | loss: 0.9036  |  0:00:14s
epoch 11 | loss: 0.81996 |  0:00:15s
epoch 12 | loss: 0.85149 |  0:00:16s
epoch 13 | loss: 0.82404 |  0:00:17s
epoch 14 | loss: 0.80869 |  0:00:19s
epoch 15 | loss: 0.81147 |  0:00:20s
epoch 16 | loss: 0.8431  |  0:00:21s
epoch 17 | loss: 0.79669 |  0:00:23s
epoch 18 | loss: 0.8514  |  0:00:25s
epoch 19 | loss: 0.84445 |  0:00:26s
epoch 20 | loss: 0.81454 |  0:00:27s
epoch 21 | loss: 0.83821 |  0:00:28s
epoch 22 | loss: 0.85655 |  0:00:29s
epoch 23 | loss: 0.87266 |  0:00:30s
epoch 24 | loss: 0.87384 |  0:00:31s
epoch 25 | loss: 0.82363 |  0:00:32s
epoch 26 | loss: 0.87253 |  0:00:33s
e

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 10.29999|  0:00:01s
epoch 1  | loss: 1.16052 |  0:00:02s
epoch 2  | loss: 0.94705 |  0:00:03s
epoch 3  | loss: 0.86259 |  0:00:04s
epoch 4  | loss: 0.81242 |  0:00:05s
epoch 5  | loss: 0.70846 |  0:00:06s
epoch 6  | loss: 0.75525 |  0:00:07s
epoch 7  | loss: 0.78905 |  0:00:09s
epoch 8  | loss: 0.74453 |  0:00:10s
epoch 9  | loss: 0.7123  |  0:00:12s
epoch 10 | loss: 0.66733 |  0:00:13s
epoch 11 | loss: 0.71848 |  0:00:14s
epoch 12 | loss: 0.66984 |  0:00:15s
epoch 13 | loss: 0.66856 |  0:00:16s
epoch 14 | loss: 0.64259 |  0:00:18s
epoch 15 | loss: 0.6697  |  0:00:19s
epoch 16 | loss: 0.63909 |  0:00:20s
epoch 17 | loss: 0.61702 |  0:00:21s
epoch 18 | loss: 0.61807 |  0:00:22s
epoch 19 | loss: 0.63118 |  0:00:24s
epoch 20 | loss: 0.60677 |  0:00:26s
epoch 21 | loss: 0.6909  |  0:00:27s
epoch 22 | loss: 0.71498 |  0:00:28s
epoch 23 | loss: 0.62698 |  0:00:29s
epoch 24 | loss: 0.6462  |  0:00:30s
epoch 25 | loss: 0.601   |  0:00:31s
epoch 26 | loss: 0.59055 |  0:00:32s
e

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


...

epoch 0  | loss: 10.21876|  0:00:01s
epoch 1  | loss: 2.65577 |  0:00:03s
epoch 2  | loss: 1.82927 |  0:00:06s
epoch 3  | loss: 1.41902 |  0:00:08s
epoch 4  | loss: 1.49402 |  0:00:11s
epoch 5  | loss: 1.25948 |  0:00:13s
epoch 6  | loss: 1.2738  |  0:00:15s
epoch 7  | loss: 1.27014 |  0:00:17s
epoch 8  | loss: 1.54701 |  0:00:19s
epoch 9  | loss: 1.32869 |  0:00:21s
epoch 10 | loss: 1.26286 |  0:00:24s
epoch 11 | loss: 1.15136 |  0:00:26s
epoch 12 | loss: 1.11751 |  0:00:28s
epoch 13 | loss: 1.09888 |  0:00:30s
epoch 14 | loss: 1.13825 |  0:00:32s
epoch 15 | loss: 1.07282 |  0:00:34s
epoch 16 | loss: 1.0843  |  0:00:37s
epoch 17 | loss: 1.11299 |  0:00:39s
epoch 18 | loss: 1.07358 |  0:00:41s
epoch 19 | loss: 1.02885 |  0:00:43s
epoch 20 | loss: 1.03714 |  0:00:45s
epoch 21 | loss: 1.06107 |  0:00:47s
epoch 22 | loss: 1.0552  |  0:00:50s
epoch 23 | loss: 1.09752 |  0:00:52s
epoch 24 | loss: 1.0533  |  0:00:54s
epoch 25 | loss: 1.13986 |  0:00:57s
epoch 26 | loss: 1.04175 |  0:00:59s
e

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 11.22081|  0:00:01s
epoch 1  | loss: 2.0856  |  0:00:03s
epoch 2  | loss: 1.69733 |  0:00:05s
epoch 3  | loss: 1.29151 |  0:00:07s
epoch 4  | loss: 1.52768 |  0:00:10s
epoch 5  | loss: 1.16419 |  0:00:13s
epoch 6  | loss: 0.992   |  0:00:15s
epoch 7  | loss: 0.90353 |  0:00:17s
epoch 8  | loss: 0.79526 |  0:00:19s
epoch 9  | loss: 0.81329 |  0:00:20s
epoch 10 | loss: 0.84324 |  0:00:23s
epoch 11 | loss: 0.76188 |  0:00:26s
epoch 12 | loss: 0.80412 |  0:00:28s
epoch 13 | loss: 0.74347 |  0:00:30s
epoch 14 | loss: 0.74744 |  0:00:32s
epoch 15 | loss: 0.71387 |  0:00:34s
epoch 16 | loss: 0.70103 |  0:00:36s
epoch 17 | loss: 0.67713 |  0:00:39s
epoch 18 | loss: 0.76664 |  0:00:41s
epoch 19 | loss: 0.70121 |  0:00:43s
epoch 20 | loss: 0.66839 |  0:00:45s
epoch 21 | loss: 0.65796 |  0:00:47s
epoch 22 | loss: 0.75698 |  0:00:49s
epoch 23 | loss: 0.8425  |  0:00:52s
epoch 24 | loss: 0.73741 |  0:00:54s
epoch 25 | loss: 0.71223 |  0:00:56s
epoch 26 | loss: 0.68961 |  0:00:58s
e

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 11.91547|  0:00:02s
epoch 1  | loss: 3.92292 |  0:00:04s
epoch 2  | loss: 2.69444 |  0:00:06s
epoch 3  | loss: 1.82479 |  0:00:08s
epoch 4  | loss: 1.65187 |  0:00:10s
epoch 5  | loss: 1.55213 |  0:00:13s
epoch 6  | loss: 1.60008 |  0:00:15s
epoch 7  | loss: 1.54712 |  0:00:17s
epoch 8  | loss: 1.75638 |  0:00:19s
epoch 9  | loss: 1.55953 |  0:00:21s
epoch 10 | loss: 1.59154 |  0:00:23s
epoch 11 | loss: 1.52209 |  0:00:26s
epoch 12 | loss: 1.46754 |  0:00:28s
epoch 13 | loss: 1.49198 |  0:00:30s
epoch 14 | loss: 1.50745 |  0:00:32s
epoch 15 | loss: 1.56514 |  0:00:34s
epoch 16 | loss: 1.76418 |  0:00:36s
epoch 17 | loss: 1.48169 |  0:00:39s
epoch 18 | loss: 1.52631 |  0:00:42s
epoch 19 | loss: 1.46732 |  0:00:44s
epoch 20 | loss: 1.59984 |  0:00:46s
epoch 21 | loss: 1.4755  |  0:00:48s
epoch 22 | loss: 1.40596 |  0:00:50s
epoch 23 | loss: 1.45197 |  0:00:53s
epoch 24 | loss: 1.43538 |  0:00:55s
epoch 25 | loss: 1.55562 |  0:00:57s
epoch 26 | loss: 1.42517 |  0:00:59s
e

In [10]:
# Tạo DataFrame kết quả
results_df = pd.DataFrame(results)
results_df.to_csv("tabnet_model_results.csv", index=False)

print(results_df)

  Target Column       MSE                                Params
0    diemtbhk_2  0.822017  {'n_a': 48, 'n_d': 48, 'n_steps': 7}
1    diemtbhk_3  0.593918  {'n_a': 48, 'n_d': 48, 'n_steps': 7}
2    diemtbhk_4  0.939546  {'n_a': 48, 'n_d': 48, 'n_steps': 7}
3    diemtbhk_5  0.569321  {'n_a': 48, 'n_d': 48, 'n_steps': 7}
4    diemtbhk_6  1.103035  {'n_a': 48, 'n_d': 48, 'n_steps': 7}
5    diemtbhk_7  0.501742  {'n_a': 48, 'n_d': 48, 'n_steps': 7}
6    diemtbhk_8  1.412368  {'n_a': 48, 'n_d': 48, 'n_steps': 7}


In [11]:
print(results_df['R2 Score'].mean())
print(results_df['MSE'].mean())
print(results_df['RMSE'].mean())
print(results_df['MAE'].mean())

KeyError: 'R2 Score'